# TSA Chapter 10: SARIMA Estimated Parameters

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10_sarima_parameters/TSA_ch10_sarima_parameters.ipynb)

This notebook demonstrates:
- Displaying the estimated parameters of the best SARIMA model fitted on unemployment data (train + validation).

In [ ]:
!pip install pandas-datareader statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas_datareader as pdr
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download unemployment data
unemp = pdr.get_data_fred('UNRATE', start='2010-01-01', end='2025-01-15')
unemp_series = unemp['UNRATE']
print(f'Data: {len(unemp_series)} observations')

# 70% / 20% / 10% split
train_end = '2020-06-01'
val_start = '2020-07-01'
val_end = '2023-06-01'
test_start = '2023-07-01'

train_data = unemp_series[unemp_series.index <= train_end]
val_data = unemp_series[(unemp_series.index >= val_start) & (unemp_series.index <= val_end)]
test_data = unemp_series[unemp_series.index >= test_start]
train_val_data = unemp_series[unemp_series.index <= val_end]

print(f'Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}')

# Fit SARIMA on train+val and display parameters
model = SARIMAX(train_val_data, order=(1,1,1), seasonal_order=(1,0,1,12),
                enforce_stationarity=False, enforce_invertibility=False)
result = model.fit(disp=False)
print(result.summary())

params = result.params
std_errors = result.bse
pvalues = result.pvalues

fig, ax = plt.subplots(figsize=(8, 3))
ax.axis('off')
table_data = [[p, f'{c:.4f}', f'{s:.4f}', f'{pv:.4f}',
               '***' if pv<0.01 else ('**' if pv<0.05 else ('*' if pv<0.1 else ''))]
              for p, c, s, pv in zip(params.index, params.values, std_errors.values, pvalues.values)]
table = ax.table(cellText=table_data,
                 colLabels=['Parameter', 'Coef', 'Std Err', 'P-value', 'Sig'],
                 cellLoc='center', loc='center', colWidths=[0.25, 0.18, 0.18, 0.18, 0.1])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)
for i in range(5):
    table[(0, i)].set_facecolor(COLORS['blue'])
    table[(0, i)].set_text_props(color='white', fontweight='bold')
ax.set_title('SARIMA(1,1,1)(1,0,1) - Parameters (Train+Val)', fontweight='bold', fontsize=12, pad=20)
fig.tight_layout()
save_chart(fig, 'sarima_parameters')
plt.show()